In [11]:
import abc
from loguru import logger as logging
import sys

class Register(abc.ABC):
    
    def __init__(self, name):
        self.__name__ = name.lower()
        self.__list = {}
    
    def __getitem__(self, name):
        name = name.lower()
        if name not in self.__list:
            logging.error(f"Name {name} not found in {self.__class__.__name__}")
            sys.exit(1) 
        else:
            return self.__list[name]
    
    def __setitem__(self, name, value):
        name = name.lower()
        self.__list[name] = value
    
    def register(self, name):
        def wrapper(cls):
            if name in self.__dict__:
                logging.error(f"Name {name} already registered")
                sys.exit(1)
            self[name] = cls
            return cls
        return wrapper
    
    @property
    def list(self):
        return list(self.__list.keys())



In [12]:
class TestRegister(Register):
    
    def __init__(self, name):
        super().__init__(name)

In [13]:
test_register = TestRegister("test_register")

@test_register.register("test1")
class Test1:
    
    def __init__(self):
        pass
    
    def test(self):
        print("Test1")

In [14]:
a = test_register["test1"]

In [15]:
print(a)

<class '__main__.Test1'>


In [16]:
a().test()

Test1


In [17]:
test_register.list

['test1']

In [1]:
import numpy
import colorsys
import random
import os
from matplotlib.mlab import PCA as mlabPCA
from matplotlib import pyplot as plt


def get_colors(num_colors):
    """
    Function to generate a list of randomly generated colors
    The function first generates 256 different colors and then
    we randomly select the number of colors required from it
    num_colors        -> Number of colors to generate
    colors            -> Consists of 256 different colors
    random_colors     -> Randomly returns required(num_color) colors
    """
    colors = []
    random_colors = []
    # Generate 256 different colors and choose num_clors randomly
    for i in numpy.arange(0., 360., 360. / 256.):
        hue = i / 360.
        lightness = (50 + numpy.random.rand() * 10) / 100.
        saturation = (90 + numpy.random.rand() * 10) / 100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))

    for i in range(0, num_colors):
        random_colors.append(colors[random.randint(0, len(colors) - 1)])
    return random_colors


def random_centroid_selector(total_clusters , clusters_plotted):
    """
    Function to generate a list of randomly selected
    centroids to plot on the output png
    total_clusters        -> Total number of clusters
    clusters_plotted      -> Number of clusters to plot
    random_list           -> Contains the index of clusters
                             to be plotted
    """
    random_list = []
    for i in range(0 , clusters_plotted):
        random_list.append(random.randint(0, total_clusters - 1))
    return random_list

def plot_cluster(kmeansdata, centroid_list, label_list , num_cluster):
    """
    Function to convert the n-dimensional cluster to 
    2-dimensional cluster and plotting 50 random clusters
    file%d.png    -> file where the output is stored indexed
                     by first available file index
                     e.g. file1.png , file2.png ...
    """
    mlab_pca = mlabPCA(kmeansdata)
    cutoff = mlab_pca.fracs[1]
    users_2d = mlab_pca.project(kmeansdata, minfrac=cutoff)
    centroids_2d = mlab_pca.project(centroid_list, minfrac=cutoff)


    colors = get_colors(num_cluster)
    plt.figure()
    plt.xlim([users_2d[:, 0].min() - 3, users_2d[:, 0].max() + 3])
    plt.ylim([users_2d[:, 1].min() - 3, users_2d[:, 1].max() + 3])

    # Plotting 50 clusters only for now
    random_list = random_centroid_selector(num_cluster , 50)

    # Plotting only the centroids which were randomly_selected
    # Centroids are represented as a large 'o' marker
    for i, position in enumerate(centroids_2d):
        if i in random_list:
            plt.scatter(centroids_2d[i, 0], centroids_2d[i, 1], marker='o', c=colors[i], s=100)


    # Plotting only the points whose centers were plotted
    # Points are represented as a small '+' marker
    for i, position in enumerate(label_list):
        if position in random_list:
            plt.scatter(users_2d[i, 0], users_2d[i, 1] , marker='+' , c=colors[position])

    filename = "name"
    i = 0
    while True:
        if os.path.isfile(filename + str(i) + ".png") == False:
            #new index found write file and return
            plt.savefig(filename + str(i) + ".png")
            break
        else:
            #Changing index to next number
            i = i + 1
    return

ImportError: cannot import name 'PCA' from 'matplotlib.mlab' (/home/hpnquoc/auto-core/.env/lib/python3.12/site-packages/matplotlib/mlab.py)

In [1]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from typing import (
    Any,
)

In [2]:
class ImageTensor(torch.Tensor):

    def __init__(self, data: Any, name: str = "image_tensor", ttype: str = "fourier"):
        super().__init__()
        self.data = data
        self.__name__ = name
        self.width = data.shape[1]
        self.height = data.shape[0]
        self.channels = data.shape[2]
        
        if ttype == "fourier":
            self.data = self.fft()
            t = self.center_crop(200)
            self.data = t.data

    def fft(self):
        r = self.data[:,:,0]
        g = self.data[:,:,1]
        b = self.data[:,:,2]

        r = torch.fft.fft2(r)
        g = torch.fft.fft2(g)
        b = torch.fft.fft2(b)

        r = torch.fft.fftshift(r)
        g = torch.fft.fftshift(g)
        b = torch.fft.fftshift(b)

        return torch.stack([r, g, b], axis=2)
        # F1 = torch.fft.fft2(self)
        # F2 = torch.fft.fftshift(F1)
        return F1
    
    # def fftshift(self):
    
    def ifft(self):
        r = self.data[:,:,0]
        g = self.data[:,:,1]
        b = self.data[:,:,2]

        r = torch.fft.ifftshift(r)
        g = torch.fft.ifftshift(g)
        b = torch.fft.ifftshift(b)

        r = torch.fft.ifft2(r)
        g = torch.fft.ifft2(g)
        b = torch.fft.ifft2(b)

        return torch.stack([r, g, b], axis=2)
        # F1 = torch.fft.ifftshift(self)
        F2 = torch.fft.ifft2(self)
        return F2
    
    def _crop(self, x1, y1, x2, y2):
        return self[x1:x2, y1:y2]
    
    def center_crop(self, size):
        x1 = (self.height - size) // 2
        y1 = (self.width - size) // 2
        x2 = x1 + size
        y2 = y1 + size
        print(x1, y1, x2, y2)

        # print(self.data.shape)
        temp = self._crop(x1, y1, x2, y2)
        # print(temp.shape)
        # temp = temp.pad(self.height, self.width)
        temp = temp.pad_dim(size, self.height, self.width)
        return temp
    
    def outer_crop(self, size):
        mask = torch.zeros(size, size, 3)
        hpad = (self.height - size) // 2
        wpad = (self.width - size) // 2
        print(hpad, wpad)
        mask = torch.nn.functional.pad(mask, (0, 0, wpad, wpad, hpad, hpad), mode='constant', value=1)

        return self.data * mask

    # def pad(self):
    #     hpad = (self.height - self.data.shape[0]) // 2
    #     wpad = (self.width - self.data.shape[1]) // 2
    #     return torch.nn.functional.pad(self, (0, 0, wpad, wpad, hpad, hpad), mode='constant', value=0)
    
    def pad(self, height, width):
        if height:
            hpad = (height - self.data.shape[0]) // 2
        else:
            hpad = 0

        if width:
            wpad = (width - self.data.shape[1]) // 2
        else:
            wpad = 0
        return torch.nn.functional.pad(self, (0, 0, wpad, wpad, hpad, hpad), mode='constant', value=0)
    
    def pad_dim(self, size, height, width):
        if height > width:
            ratio = height / width
            newh = int(size*ratio)
            temp = self.pad(newh, 0)
        else:
            ratio = width / height
            neww = int(size*ratio)
            temp = self.pad(0, neww)
        return temp


    

In [3]:

# Load the image with OpenCV
image_path = 'dummy/images/4.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Convert the image to a PyTorch tensor and normalize it to range [0, 1]
image_tensor = torch.tensor(image, dtype=torch.float32) / 255.0

# Compute the 2D FFT
# fft_image = torch.fft.fft2(image_tensor)

# Shift the zero frequency component to the center
fft_shifted = ImageTensor(image_tensor)
# print(fft_shifted.shape)

# Compute the magnitude spectrum and use logarithmic scale for better visualization
magnitude_spectrum = torch.log(1 + torch.abs(fft_shifted))

# Convert the magnitude spectrum to a NumPy array and normalize it for display with OpenCV
magnitude_spectrum_np = magnitude_spectrum.numpy()
magnitude_spectrum_normalized = cv2.normalize(magnitude_spectrum_np, None, 0, 255, cv2.NORM_MINMAX)
magnitude_spectrum_display = magnitude_spectrum_normalized.astype(np.uint8)

# Display the original image and its FFT magnitude spectrum using OpenCV
cv2.imshow('Original Image', image)
cv2.imshow('FFT Magnitude Spectrum', magnitude_spectrum_display)

iff_image = fft_shifted.ifft()
# print(iff_image)
iff_image = torch.abs(iff_image)
# print(iff_image)
# Normalize the IFFT image for display with OpenCV
iff_image = iff_image.numpy()

# iff_image = cv2.normalize(iff_image, None, 0, 255, cv2.NORM_MINMAX)
# fft_img = np.concatenate((magnitude_spectrum_display, iff_image), axis=0)
cv2.imshow('IFFT Image', iff_image)

print(image.shape)
print(magnitude_spectrum_display.shape)
print(iff_image.shape)
# Wait for a key press and close the windows
cv2.waitKey(0)
cv2.destroyAllWindows()


100 200 300 400
(400, 600, 3)
(200, 300, 3)
(200, 300, 3)


In [4]:
iff_image = cv2.resize(iff_image, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_CUBIC)

In [5]:
image = image_tensor.numpy()
cv2.imshow('IFFT Image', image + 5*image*(iff_image)+0.1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import subprocess

# Spawn a process to execute a command
process = subprocess.Popen(['python', '-c', 'print(input("Enter your name: "))'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)

# Send user input to the process's stdin
process.stdin.write(b'John Doe\n')
process.stdin.flush()

# Read the output from stdout and stderr
output, error = process.communicate()

# Print the output and error messages
print(output.decode())
print(error.decode())

Enter your name: John Doe




In [6]:
from subprocess import run, PIPE

p = run(['grep', 'f'], stdout=PIPE,
        input='one\ntwo\nthree\nfour\nfive\nsix\n', encoding='ascii')
print(p.returncode)
# -> 0
print(p.stdout)

0
four
five



In [23]:
import subprocess
import os
read, write = os.pipe()
os.write(write, b"aaa\n")
os.close(write)
process = subprocess.Popen(['grep','f'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
process.stdin.write(b'one\ntwo\nthree\nfour\nfive\nsix\n')

# output, error = process.communicate()[0]
print(process.communicate()[0])

# Print the output and error messages
# print(output.decode())
# print(error.decode())

b'four\nfive\n'


In [32]:
import subprocess

process = subprocess.Popen(['python','test.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
process.stdin.write(b'one\ntwo\nthree\nfour\nfive\nsix\n')

# output, error = process.communicate()[0]
print(process.communicate()[0])

b'one\ntwo\nthree\nfour\nfive\nsix\n'
